In [ ]:
from docx import Document
import docx2txt
from pymongo import MongoClient
import re
import json

# MongoDB 연결
# client = MongoClient("")  
# db = client["testDB"]
# collection = db["industry_metrics"]

# 워드 문서에서 텍스트 전체 추출,
file_path = "산업분석.docx"
full_text = docx2txt.process(file_path)

# 블록 분리: "/ 산업명" 기준으로 분리,
blocks = re.split(r"\n/\s*", full_text)
if blocks[0].strip() == "":
    blocks = blocks[1:]

# 산업 블록별로 저장,
docs = []
for block in blocks:
    lines = block.strip().splitlines()
    if not lines:
        continue

    # 첫 줄: 산업명
    industry_name = lines[0].strip().lstrip("/").strip()

    # 나머지 줄: 전체 분석 텍스트
    analysis_text = "\n".join(lines[1:]).strip()

    doc = {
        "industry": industry_name,
        "analysis": analysis_text
    }
    docs.append(doc)

# # DB 초기화 후 저장,
# collection.delete_many({})
# collection.insert_many(docs)
# print(f" 업로드 완료: {len(docs)}개 산업 분석 저장 완료")


#json파일로 저장하기
# with open("industry_analysis.json", "w", encoding="utf-8") as f:
#     json.dump(docs, f, ensure_ascii=False, indent=2)

In [15]:
for i in docs:
    print(i)

{'industry': 'IT 서비스', 'analysis': '📡 IT 서비스 산업에서의 주요 재무 지표 해석\n\n1. PER (주가수익비율)\n\n산업 평균: 한국 상장사 기준으로 25~35배 정도가 일반적이야.\n\n해석:\n\n일반 제조업보다 PER이 높은 편인데, 고성장 기대감이 반영된 결과지.\n\n특히 클라우드, AI, B2B SaaS 기반이면 PER 40배 넘어도 투자자들이 감내하더라고.\n\n내가 예전에 말했지, "High P/E ratios are justified only by high growth and durable advantages."\n성장률이 실적을 못 받쳐주면, 고평가된 주식은 고층 건물 위에서 줄타기하는 셈이야.\n\n\n\n2. ROE (자기자본이익률)\n\n산업 평균: 10~15% 이상이면 우수한 편.\n\n특징: 고정자산이 적고, 사람 중심의 서비스 산업이라 자기자본 효율이 꽤 좋아.\n\n클라우드 기반으로 넘어간 기업은 20%대 ROE도 보여.\n\nROE는 마치 기업의 \'근육량\' 같은 거야. 마른 몸에 근육이 잘 잡혀 있으면 강하다는 뜻이거든.\n\n\n\n3. PBR (주가순자산비율)\n\n산업 평균: 3~5배 정도가 일반적이야.\n\n포인트: 유형자산보다 무형자산(브랜드, 인력, 코드, IP) 비중이 높아서 PBR로는 제대로 평가하기 어렵지.\n\n그래서 PBR이 높더라도 무조건 ‘고평가’라고 단정하면 안 돼.\n\n\n\n4. 부채비율\n\n산업 평균: 50% 미만이 바람직한 수준.\n\n특징: 자본집약적 산업이 아니라 부채가 많은 구조는 아냐.\n\n오히려 현금이 쌓이는 구조가 많고, 부채보다 선수금이나 수주계약이 많은 편이지.\n\n\n\n5. 영업이익률\n\n산업 평균: **10~20%**면 매우 우수.\n\n클라우드, SaaS 기업은 30%도 가능하지만,\nSI(시스템 통합) 위주 기업은 5~10% 수준에 머무는 경우도 있어.\n\n“When you look at a business, focus on

In [ ]:
import pandas as pd
from pymongo import MongoClient
import json

# CSV 또는 Excel 파일 로드
with open("industry_analysis_renamed.json", "r", encoding="utf-8") as file:
    data = json.load(file)


# MongoDB 연결
client = MongoClient("")
db = client["testDB"]
collection = db["industry_1"]


# 2. JSON 파일 읽기
with open("industry_analysis_renamed.json", "r", encoding="utf-8") as file:
    data = json.load(file)

collection.delete_many({})

# 3. MongoDB에 업로드 (list 형태로 넣기)
if isinstance(data, list):
    collection.insert_many(data)
else:
    collection.insert_one(data)

print("✅ MongoDB 업로드 완료!")

✅ MongoDB 업로드 완료!


In [ ]:
for i in collection.find():
    print(i)

{'_id': ObjectId('682c7356ee40e59b083fd694'), 'industry': 'IT 서비스', 'analysis': {'📡 IT 서비스 산업에서의 주요 재무 지표 해석': {'PER (주가수익비율)': {'산업 평균': '한국 상장사 기준으로 25~35배 정도가 일반적', '해석': '제조업보다 PER이 높은 편인데, 고성장 기대감이 반영된 결과지.\n\n특히 클라우드, AI, B2B SaaS 기반이면 PER 40배 넘어도 투자자들이 감내하더라고.\n\n내가 예전에 말했지, "High P/E ratios are justified only by high growth and durable advantages."\n성장률이 실적을 못 받쳐주면, 고평가된 주식은 고층 건물 위에서 줄타기하는 셈이야.'}, 'ROE (자기자본이익률)': {'산업 평균': '10~15% 이상이면 우수한 편', '해석': "고정자산이 적고, 사람 중심의 서비스 산업이라 자기자본 효율이 꽤 좋아.\n\n클라우드 기반으로 넘어간 기업은 20%대 ROE도 보여.\n\nROE는 마치 기업의 '근육량' 같은 거야. 마른 몸에 근육이 잘 잡혀 있으면 강하다는 뜻이거든."}, 'PBR (주가순자산비율)': {'산업 평균': '3~5배 정도가 일반적', '해석': '포인트: 유형자산보다 무형자산(브랜드, 인력, 코드, IP) 비중이 높아서 PBR로는 제대로 평가하기 어렵지.\n\n그래서 PBR이 높더라도 무조건 ‘고평가’라고 단정하면 안 돼.'}, '부채비율': {'산업 평균': '50% 미만이 바람직한 수준', '해석': '자본집약적 산업이 아니라 부채가 많은 구조는 아냐.\n\n오히려 현금이 쌓이는 구조가 많고, 부채보다 선수금이나 수주계약이 많은 편이지.'}, '영업이익률': {'산업 평균': '10~20% 면 매우 우수', '해석': '클라우드, SaaS 기업은 30%도 가능하지만,\nSI(시스템 통합) 위주 기업은 5~10% 수준에 머무는 경우도 있어.\n\n“When

In [7]:
#key수정
import json

def rename_keys_in_dict(d):
    if not isinstance(d, dict):
        return d

    new_dict = {}
    for k, v in d.items():
        # key에 '주요 재무' 포함되면 변경
        if "주요 재무" in k:
            new_key = "주요 재무 지표 해석"
        else:
            new_key = k

        # 값이 dict면 재귀 호출
        if isinstance(v, dict):
            new_dict[new_key] = rename_keys_in_dict(v)
        elif isinstance(v, list):
            new_dict[new_key] = [rename_keys_in_dict(i) if isinstance(i, dict) else i for i in v]
        else:
            new_dict[new_key] = v

    return new_dict


# JSON 파일 읽기
with open("processed_industry_analysis.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 전체 데이터 순회하며 key명 변경
for item in data:
    if "analysis" in item:
        item["analysis"] = rename_keys_in_dict(item["analysis"])

# 결과를 새 파일로 저장
with open("industry_analysis_renamed.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("완료! '주요 재무' 포함 key명이 모두 '주요 재무 지표 해석'로 변경되었습니다.")


완료! '주요 재무' 포함 key명이 모두 '주요 재무 지표 해석'로 변경되었습니다.
